[View in Colaboratory](https://colab.research.google.com/github/leandrohmvieira/ML-Capstone-Project/blob/master/Deep_fashion.ipynb)

# Aplicação de redes convolutivas no dataset deep fashion

Será construída uma rede neural convolutiva simples neste exemplo

In [1]:
#agrupa e importa todas as bibliotecas necessárias para a execução do bagulho
import keras
from keras.datasets import cifar10
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import glob
from PIL import Image

Using TensorFlow backend.


# Deep fashion dataset

O dataset utilizado para o treinamento é o Deep Fashion

In [0]:
#caso necessário, todos os arquivos referentes ao dataset Deep Fashion estão localizados abaixo
#wget https://www.dropbox.com/sh/ryl8efwispnjw21/AABpzYsttt7DIQmb2PckgbPXa/Anno/list_attr_cloth.txt?dl=0 -O fashion_data/Anno/list_attr_cloth.txt
#wget https://www.dropbox.com/sh/ryl8efwispnjw21/AADYszB-Pv6mgwtiPEtQkHTva/Anno/list_attr_img.txt?dl=0 -O fashion_data/Anno/list_attr_img.txt
#wget https://www.dropbox.com/sh/ryl8efwispnjw21/AADr1hf3nsOZEV3sOTZ1-m98a/Anno/list_bbox.txt?dl=0 -O fashion_data/Anno/list_bbox.txt
#wget https://www.dropbox.com/sh/ryl8efwispnjw21/AACiFqyjpb21GyVwLNBATFQXa/Anno/list_category_cloth.txt?dl=0 -O fashion_data/Anno/list_category_cloth.txt
#wget https://www.dropbox.com/sh/ryl8efwispnjw21/AAD3Mm6b2e9vkVdb35OfCA3fa/Anno/list_category_img.txt?dl=0 -O fashion_data/Anno/list_category_img.txt
#wget https://www.dropbox.com/sh/ryl8efwispnjw21/AAARD4rdUT8oBQsjl4HuYAXha/Anno/list_landmarks.txt?dl=0 -O fashion_data/Anno/list_landmarks.txt
#wget https://www.dropbox.com/sh/ryl8efwispnjw21/AACTJyCl9bprY90Z3frUZ-H-a/Eval/list_eval_partition.txt?dl=0 -O fashion_data/Eval/list_eval_partition.txt
#wget https://www.dropbox.com/sh/ryl8efwispnjw21/AABKePZxbIrUHD0RjFLGA9q1a/README.txt?dl=0 -O fashion_data/README.txt
#wget -c https://www.dropbox.com/sh/ryl8efwispnjw21/AACpZU-UKs_snxFH5Bp8RwOwa/Img/img.zip?dl=0 -O fashion_data/Img/img.zip

In [0]:
!mkdir -p 'downloads'

#list_category_cloth contém 50 classes de roupas a serem classificadas
!wget https://www.dropbox.com/sh/ryl8efwispnjw21/AACiFqyjpb21GyVwLNBATFQXa/Anno/list_category_cloth.txt?dl=0 -O downloads/list_category_cloth.txt

#list_eval_partition descreve quais imagens devem ser utilizadas como treinamento, validação e teste
!wget https://www.dropbox.com/sh/ryl8efwispnjw21/AACTJyCl9bprY90Z3frUZ-H-a/Eval/list_eval_partition.txt?dl=0 -O downloads/list_eval_partition.txt
  
#list_category_img contém as marcações das 50 classes em cada imagem do conjunto
!wget https://www.dropbox.com/sh/ryl8efwispnjw21/AAD3Mm6b2e9vkVdb35OfCA3fa/Anno/list_category_img.txt?dl=0 -O downloads/list_category_img.txt

#img.zip contém as imagens em si
!wget -c https://www.dropbox.com/sh/ryl8efwispnjw21/AACpZU-UKs_snxFH5Bp8RwOwa/Img/img.zip?dl=0 -O downloads/img.zip


In [0]:
#dezipando as imagens, esta etapa pode dar erro
!unzip -n downloads/img.zip -d downloads/

In [0]:
!rm -rf /content/train/
!rm -rf /content/val/
!rm -rf /content/test/

In [0]:
#separa as imagens entre treino, validação e teste

from shutil import copy
import collections
import os

file_dir = '/content/downloads/'
steps = ['train', 'val', 'test']
categories = dict()
img_params = collections.defaultdict(list)


for i in steps:
  os.makedirs(i,exist_ok=True)
  
#monta um dict com as categorias a serem classificadas
with open('downloads/list_category_cloth.txt') as f:
  next(f)
  next(f)
  lineno = 1
  for line in f:
    arglist = ' '.join(line.split()).split(' ')
    categories.update({lineno:arglist[0]})
    lineno +=1

#cria os diretórios de cada categoria nos diretórios de cada etapa
for step in steps:
  for cat in categories.keys():
    os.makedirs(step+"/"+str(cat),exist_ok=True)
    
    
#monta um dict que indica para cada arquivo, a qual etapa ele pertence e qual sua label correspondente
with open('downloads/list_eval_partition.txt') as f:
  next(f)
  next(f)
  for line in f:
    arglist = ' '.join(line.split()).split(' ')
    img_params[arglist[0]].append(arglist[1])
    
with open('downloads/list_category_img.txt') as g:
      next(g)
      next(g)
      for line2 in g:
        arglist2 = ' '.join(line2.split()).split(' ')
        img_params[arglist2[0]].append(arglist2[1])

#com os parametros necessários, copia os arquivos para seus respectivos lugares
# ESSA PORRA AQUI NÃO TÁ COPIANDO O CARALHO DAS IMAGENS
for image in img_params.keys():
  params = img_params.get(image)
  #copy2(file_dir+image,"/content/"+params[0]+"/"+str(params[1]))
  source = file_dir+image
  dest = "/content/"+params[0]+'/'+str(params[1])+'/'+image.split('/')[1]+image.split('/')[2]
  copy(source,dest)
  #print("copiando {} para {}".format(source,dest))

In [75]:
total_files = len(glob.glob('/content/downloads/img/*/*.jpg'))
total_train = len(glob.glob('/content/train/*/*.jpg'))
total_val = len(glob.glob('/content/val/*/*.jpg'))
total_test = len(glob.glob('/content/test/*/*.jpg'))
 
print('total de arquivos do dataset:{}'.format(total_files))
print('total de arquivos de treino:{}'.format(total_train))
print('total de arquivos de validacao:{}'.format(total_val))
print('total de arquivos de teste:{}'.format(total_test))

  

total de arquivos do dataset:289229
total de arquivos de treino:209222
total de arquivos de validacao:40000
total de arquivos de teste:40000


# Preprocessing

Agora que os arquivos estão dispostos nas pastas , iremos montar as entradas para o treinamento da rede neural. O Keras não aceita imagens com resoluções variadas, para contornar estas situações,como primeira medida irei redimensionar todas as imagens para o tamanho da maior imagem do conjunto.

Caso o redimensionamento das imagens não deem um bom resultado, tentarei métodos mais sofisticados.

In [0]:
#usarei a classe ImageDataGenerator para corrigir as diferenças de resolução entre as imagens

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

In [0]:
#Para utilizar o gerador de dados é necessário inferir os dados de altura e largura da imagem

import cv2


In [89]:
#A partir dos geradores é possível chamar a função flow_from_directory, que cria o objeto de dados a partir de um path

train_generator = train_datagen.flow_from_directory(
    '/content/train/',
    target_size=(300,200),
    batch_size=64,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    '/content/val/',
    target_size=(300,200),
    batch_size=64,
    class_mode='categorical'
)

Found 209222 images belonging to 50 classes.
Found 40000 images belonging to 50 classes.


# Definindo a arquitetura da Rede Neural

A arquitetura inicial consiste em treinar uma rede convolucional com 3 camadas de convolucao, separadas por camadas de max pooling, encerrando em 2 camadas densas

In [90]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu', 
                        input_shape=(300, 200, 3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(50, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 300, 200, 16)      208       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 150, 100, 16)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 150, 100, 32)      2080      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 75, 50, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 75, 50, 64)        8256      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 37, 25, 64)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 37, 25, 64)        0         
__________

In [0]:
#compila e treina o modelo

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', 
                  metrics=['accuracy'])

In [0]:
from keras.callbacks import ModelCheckpoint 

# train the model
checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5', verbose=1, 
                               save_best_only=True)

model.fit_generator(train_generator,validation_data=val_generator,epochs=2,callbacks=[checkpointer])

Epoch 1/2
 561/3270 [====>.........................] - ETA: 11:06 - loss: 3.0039 - acc: 0.3108

In [0]:
from google.colab import files

with open('example.txt', 'w') as f:
  f.write('some content')

files.download('example.txt')